# 2.7 学習と検証の実施

- 本ファイルでは、SSDの学習と検証の実施を行います。手元のマシンで動作を確認後、AWSのGPUマシンで計算します。
- p2.xlargeで約6時間かかります。


# 学習目標

1.	SSDの学習を実装できるようになる

# 事前準備

- AWS EC2 のGPUインスタンスを使用します
- フォルダ「utils」のssd_model.pyをします

In [19]:
# パッケージのimport
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [20]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

使用デバイス： cpu


# DatasetとDataLoaderを作成する

In [22]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn


# ファイルパスのリストを取得
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# Datasetを作成
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoaderを作成する
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# ネットワークモデルの作成する

In [23]:
from utils.ssd_model import SSD

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="train", cfg=ssd_cfg)

# SSDの初期の重みを設定
# ssdのvgg部分に重みをロードする
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssdのその他のネットワークの重みはHeの初期値で初期化


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)


# Heの初期値を適用
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

print('ネットワーク設定完了：学習済みの重みをロードしました')


使用デバイス： cpu
ネットワーク設定完了：学習済みの重みをロードしました


# 損失関数と最適化手法を定義する

In [24]:
from utils.ssd_model import MultiBoxLoss

# 損失関数の設定
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 最適化手法の設定
optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)


# 学習・検証を実施する

In [25]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    logs = []

    # epochのループ
    for epoch in range(num_epochs+1):

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('（train）')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('（val）')
                else:
                    # 検証は10回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            for images, targets in dataloaders_dict[phase]:

                # GPUが使えるならGPUにデータを送る
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # リストの各要素のテンソルをGPUへ

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）計算
                    outputs = net(images)

                    # 損失の計算
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算

                        # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # パラメータ更新

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのloss （Issue158での誤植修正）
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        # ネットワークを保存する
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')


In [26]:
# 学習・検証を実行する
num_epochs= 50  
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cpu
-------------
Epoch 1/50
-------------
（train）
イテレーション 10 || Loss: 16.5946 || 10iter: 238.6400 sec.
イテレーション 20 || Loss: 12.9359 || 10iter: 229.9261 sec.
イテレーション 30 || Loss: 10.3333 || 10iter: 216.5402 sec.
イテレーション 40 || Loss: 8.8764 || 10iter: 216.8771 sec.
イテレーション 50 || Loss: 8.0001 || 10iter: 215.1483 sec.
イテレーション 60 || Loss: 7.9889 || 10iter: 217.0604 sec.
イテレーション 70 || Loss: 8.2060 || 10iter: 215.9383 sec.
イテレーション 80 || Loss: 7.4541 || 10iter: 215.9049 sec.
イテレーション 90 || Loss: 7.9851 || 10iter: 215.8946 sec.
イテレーション 100 || Loss: 7.6231 || 10iter: 215.6451 sec.
イテレーション 110 || Loss: 7.6616 || 10iter: 216.2955 sec.
イテレーション 120 || Loss: 7.1604 || 10iter: 215.0701 sec.
イテレーション 130 || Loss: 7.6439 || 10iter: 214.8962 sec.
イテレーション 140 || Loss: 7.5633 || 10iter: 215.6857 sec.
イテレーション 150 || Loss: 7.2468 || 10iter: 215.2398 sec.
イテレーション 160 || Loss: 7.6056 || 10iter: 215.6271 sec.
イテレーション 170 || Loss: 7.1819 || 10iter: 217.3203 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:1625

/Users/sadahiroyoshi/Documents/apps/mygithub/pytorch_advanced/2_objectdetection/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


イテレーション 180 || Loss: 7.4451 || 10iter: 21.8092 sec.
イテレーション 190 || Loss: 7.2548 || 10iter: 216.0269 sec.
イテレーション 200 || Loss: 7.0377 || 10iter: 215.0914 sec.
イテレーション 210 || Loss: 6.6003 || 10iter: 215.8473 sec.
イテレーション 220 || Loss: 6.9190 || 10iter: 215.1952 sec.
イテレーション 230 || Loss: 7.1063 || 10iter: 215.8553 sec.
イテレーション 240 || Loss: 7.0255 || 10iter: 215.1076 sec.
イテレーション 250 || Loss: 7.0571 || 10iter: 216.2073 sec.
イテレーション 260 || Loss: 6.8714 || 10iter: 216.0279 sec.
イテレーション 270 || Loss: 7.1949 || 10iter: 214.8178 sec.
イテレーション 280 || Loss: 6.6342 || 10iter: 215.1923 sec.
イテレーション 290 || Loss: 6.5217 || 10iter: 214.3552 sec.
イテレーション 300 || Loss: 6.2998 || 10iter: 214.8561 sec.
イテレーション 310 || Loss: 6.8146 || 10iter: 213.5292 sec.
イテレーション 320 || Loss: 6.6065 || 10iter: 215.2885 sec.
イテレーション 330 || Loss: 7.0427 || 10iter: 215.4124 sec.
イテレーション 340 || Loss: 6.3437 || 10iter: 216.7173 sec.
イテレーション 350 || Loss: 6.8867 || 10iter: 214.2997 sec.
-------------
epoch 2 || Epoch_TRAIN_Loss:1248.

/Users/sadahiroyoshi/Documents/apps/mygithub/pytorch_advanced/2_objectdetection/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


イテレーション 360 || Loss: 6.6364 || 10iter: 42.7014 sec.
イテレーション 370 || Loss: 6.5726 || 10iter: 213.5779 sec.
イテレーション 380 || Loss: 6.5111 || 10iter: 215.1523 sec.
イテレーション 390 || Loss: 6.6028 || 10iter: 215.5877 sec.
イテレーション 400 || Loss: 7.5662 || 10iter: 215.3996 sec.
イテレーション 410 || Loss: 6.5893 || 10iter: 214.2125 sec.
イテレーション 420 || Loss: 6.5986 || 10iter: 214.4445 sec.
イテレーション 430 || Loss: 6.4346 || 10iter: 215.3139 sec.
イテレーション 440 || Loss: 6.7460 || 10iter: 215.2642 sec.
イテレーション 450 || Loss: 6.6850 || 10iter: 215.0733 sec.
イテレーション 460 || Loss: 6.4776 || 10iter: 214.8917 sec.
イテレーション 470 || Loss: 6.5756 || 10iter: 215.5516 sec.
イテレーション 480 || Loss: 6.4453 || 10iter: 214.4369 sec.
イテレーション 490 || Loss: 6.6973 || 10iter: 214.9562 sec.
イテレーション 500 || Loss: 6.3156 || 10iter: 215.4299 sec.
イテレーション 510 || Loss: 6.2098 || 10iter: 216.7683 sec.
イテレーション 520 || Loss: 6.5111 || 10iter: 214.0202 sec.
イテレーション 530 || Loss: 6.3186 || 10iter: 214.7340 sec.
-------------
epoch 3 || Epoch_TRAIN_Loss:1173.

...

イテレーション 3230 || Loss: 4.2279 || 10iter: 179.6365 sec.
イテレーション 3240 || Loss: 4.0767 || 10iter: 222.4386 sec.
イテレーション 3250 || Loss: 4.6060 || 10iter: 221.7786 sec.
イテレーション 3260 || Loss: 4.1811 || 10iter: 236.4171 sec.
イテレーション 3270 || Loss: 4.3967 || 10iter: 237.9147 sec.
イテレーション 3280 || Loss: 4.5125 || 10iter: 220.7788 sec.
イテレーション 3290 || Loss: 4.2316 || 10iter: 226.4430 sec.
イテレーション 3300 || Loss: 4.8699 || 10iter: 216.8205 sec.
イテレーション 3310 || Loss: 4.6037 || 10iter: 215.3679 sec.
イテレーション 3320 || Loss: 4.6456 || 10iter: 215.4122 sec.
イテレーション 3330 || Loss: 3.7827 || 10iter: 213.0359 sec.
イテレーション 3340 || Loss: 4.5761 || 10iter: 211.0557 sec.
イテレーション 3350 || Loss: 4.9560 || 10iter: 213.0929 sec.
イテレーション 3360 || Loss: 4.3570 || 10iter: 213.4968 sec.
イテレーション 3370 || Loss: 3.7871 || 10iter: 212.8679 sec.
イテレーション 3380 || Loss: 4.2252 || 10iter: 216.2275 sec.
イテレーション 3390 || Loss: 4.2542 || 10iter: 212.3776 sec.
イテレーション 3400 || Loss: 4.6654 || 10iter: 212.3638 sec.
-------------
epoch 19 || Ep

/Users/sadahiroyoshi/Documents/apps/mygithub/pytorch_advanced/2_objectdetection/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


イテレーション 3410 || Loss: 4.4441 || 10iter: 190.9428 sec.
イテレーション 3420 || Loss: 4.4974 || 10iter: 212.5516 sec.
イテレーション 3430 || Loss: 4.0508 || 10iter: 211.4168 sec.
イテレーション 3440 || Loss: 4.3461 || 10iter: 230.9340 sec.
イテレーション 3450 || Loss: 4.1018 || 10iter: 230.0077 sec.
イテレーション 3460 || Loss: 4.2959 || 10iter: 219.9004 sec.
イテレーション 3470 || Loss: 4.5836 || 10iter: 220.0517 sec.
イテレーション 3480 || Loss: 4.9729 || 10iter: 217.4121 sec.
イテレーション 3490 || Loss: 4.3946 || 10iter: 214.8045 sec.
イテレーション 3500 || Loss: 4.6895 || 10iter: 235.7880 sec.
イテレーション 3510 || Loss: 4.8740 || 10iter: 238.4052 sec.
イテレーション 3520 || Loss: 4.0771 || 10iter: 233.7517 sec.
イテレーション 3530 || Loss: 4.1755 || 10iter: 214.1058 sec.
イテレーション 3540 || Loss: 4.6049 || 10iter: 213.5697 sec.
イテレーション 3550 || Loss: 4.7765 || 10iter: 221.4950 sec.
イテレーション 3560 || Loss: 3.9749 || 10iter: 229.8259 sec.
イテレーション 3570 || Loss: 4.6201 || 10iter: 231.6271 sec.
イテレーション 3580 || Loss: 4.6185 || 10iter: 233.2474 sec.
-------------
（val）
--------

/Users/sadahiroyoshi/Documents/apps/mygithub/pytorch_advanced/2_objectdetection/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


イテレーション 3590 || Loss: 4.5889 || 10iter: 267.3543 sec.


以上